# Tracing and Datasets with LangChainPlus

LangChain makes it easy to get started with Agents and other LLM applications. However, it can be tricky to get right, especially when you need to deliver a full product. To speed up your application development process, and to help monitor your applications in production, LangChain offers additional tracing and tooling.

When might you want to use tracing? Some situations we've found it useful include:
- Quickly debugging a new chain, agent, or set of tools
- Evaluating a given chain across different LLMs or Chat Models to compare results or improve prompts
- Running a given chain multiple time on a dataset to ensure it consistently meets a quality bar.


In this notebook, we'll show how to enable tracing in your LangChain applications and walk you a couple common ways to evaluate your agents.
We'll focus on using Datasets to benchmark Chain behavior.

**Bear in mind that this notebook is designed under the assumption that you're running the latest LangChain+ server locally in the background. This is done using the folowing command in your terminal:**


```
pip install --upgrade langchain
langchain plus start
```

We also have a hosted version which is in private beta. We will share more details as it progresses.

Now, let's get started by creating a client to connect to LangChain+.

## Setting up Tracing

The V2 tracing API can be activated by setting the `LANGCHAIN_TRACING_V2` environment variable to true. Assuming you've successfully initiated the server as described earlier, running LangChain Agents, Chains, LLMs, and other primitives will automatically start capturing traces. Let's begin our exploration with a straightforward math example.

**NOTE**: You must also set your `OPENAI_API_KEY` and `SERPAPI_API_KEY` environment variables in order to run the following tutorial.


**NOTE:** You can also use the `tracing_v2_enabled` context manager to capture sessions within a given context:
```
from langchain.callbacks.manager import tracing_v2_enabled
with tracing_v2_enabled("My Session Name"):
    ...
```

**NOTE:** You can optionally set the `LANGCHAIN_ENDPOINT` and `LANGCHAIN_API_KEY` environment variables if using the hosted version which is in private beta.

In [2]:
import os
from langchain.client import LangChainPlusClient

import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_SESSION"] = "Tracing Walkthrough"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://langchainpro-api-gateway-12bfv6cf.uc.gateway.dev"  # Uncomment this line if you want to use the hosted version
# os.environ["LANGCHAIN_API_KEY"] = "<YOUR-LANGCHAINPLUS-API-KEY>"  # Uncomment this line if you want to use the hosted version.

client = LangChainPlusClient()
print("You can click the link below to view the UI")
client

You can click the link below to view the UI


LangChainPlusClient (API URL: http://localhost:8000)

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

llm = ChatOpenAI(temperature=0)
tools = load_tools(['serpapi', 'llm-math'], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False)

In [4]:
import asyncio

inputs = [
    "How many people live in canada as of 2023?",
    "who is dua lipa's boyfriend? what is his age raised to the .43 power?",
    "what is dua lipa's boyfriend age raised to the .43 power?",
    "how far is it from paris to boston in miles",
    "what was the total number of points scored in the 2023 super bowl? what is that number raised to the .23 power?",
    "what was the total number of points scored in the 2023 super bowl raised to the .23 power?",
    "how many more points were scored in the 2023 super bowl than in the 2022 super bowl?",
    "what is 153 raised to .1312 power?",
    "who is kendall jenner's boyfriend? what is his height (in inches) raised to .13 power?",
    "what is 1213 divided by 4345?",
]
results = []

async def arun(agent, input_example):
    try:
        return await agent.arun(input_example)
    except Exception as e:
        # The agent sometimes makes mistakes! These will be captured by the tracing.
        print(e)
        return e
for input_example in inputs:
    results.append(arun(agent, input_example))
await asyncio.gather(*results)     

unknown format from LLM: Sorry, I cannot answer this question as it requires information from the future.
LLMMathChain._evaluate("
(age ** 0.43)
") raised error: 'age'. Please try again with a valid numerical expression
LLMMathChain._evaluate("
(total number of points scored in the 2023 super bowl)**0.23
") raised error: invalid syntax. Perhaps you forgot a comma? (<expr>, line 1). Please try again with a valid numerical expression
LLMMathChain._evaluate("
round(0.2791714614499425, 2)
") raised error: 'VariableNode' object is not callable. Please try again with a valid numerical expression


Retrying langchain.chat_models.openai.acompletion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} <CIMultiDictProxy('Date': 'Wed, 24 May 2023 02:33:01 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7cc219f30c1d421c-EWR', 'alt-svc': 'h3=":443"; ma=86400, h3-29=":443"; ma=86400')>.


['39,566,248',
 "Anwar Hadid's age raised to the 0.43 power is approximately 3.87.",
 ValueError('LLMMathChain._evaluate("\n(age ** 0.43)\n") raised error: \'age\'. Please try again with a valid numerical expression'),
 'The distance between Paris and Boston is 3448 miles.',
 ValueError('unknown format from LLM: Sorry, I cannot answer this question as it requires information from the future.'),
 ValueError('LLMMathChain._evaluate("\n(total number of points scored in the 2023 super bowl)**0.23\n") raised error: invalid syntax. Perhaps you forgot a comma? (<expr>, line 1). Please try again with a valid numerical expression'),
 '3 points were scored more in the 2023 Super Bowl than in the 2022 Super Bowl.',
 '1.9347796717823205',
 '1.2600077141429156',
 ValueError('LLMMathChain._evaluate("\nround(0.2791714614499425, 2)\n") raised error: \'VariableNode\' object is not callable. Please try again with a valid numerical expression')]

## Creating the Dataset

Now that you've captured a session entitled 'Tracing Walkthrough', it's time to create a dataset. We will do so using the `create_dataset` method below.

In [5]:
dataset_name = "calculator-example-dataset"

In [6]:
if dataset_name in set([dataset.name for dataset in client.list_datasets()]):
    client.delete_dataset(dataset_name=dataset_name)
dataset = client.create_dataset(dataset_name, description="A calculator example dataset")
runs = client.list_runs(
    session_name=os.environ["LANGCHAIN_SESSION"],
    execution_order=1, # Only return the top-level runs
    error=False, # Only runs that succeed
)
for run in runs:
    client.create_example(inputs=run.inputs, outputs=run.outputs, dataset_id=dataset.id)

**Alternative: Creating a Dataset in the UI** 

Alternatively, you could create or edit the dataset in the UI using the following steps:

   1. Navigate to the UI by clicking on the link below.
   2. Select the 'search_and_math_chain' session from the list.
   3. Next to the fist example, click "+ to Dataset".
   4. Click "Create Dataset" and create a title **"calculator-example-dataset"**.
   5. Add the other examples to the dataset as well

Once you've used LangChain+ for a while, you will have a number of datasets to work with. To view all saved datasets, execute the following code:

```
datasets = client.list_datasets()
print(datasets)
```


**Optional:** If you didn't run the trace above, you can also create datasets by uploading dataframes or CSV files.

In [7]:
# !pip install datasets > /dev/null
# !pip install pandas > /dev/null

In [8]:
# import pandas as pd
# from langchain.evaluation.loading import load_dataset

# dataset = load_dataset("agent-search-calculator")
# df = pd.DataFrame(dataset, columns=["question", "answer"])
# df.columns = ["input", "output"] # The chain we want to evaluate below expects inputs with the "input" key 
# df.head()

In [9]:
# dataset_name = "calculator-example-dataset"

# if dataset_name not in set([dataset.name for dataset in client.list_datasets()]):
#     dataset = client.upload_dataframe(df, 
#                             name=dataset_name,
#                             description="A calculator example dataset",
#                             input_keys=["input"],
#                             output_keys=["output"],
#                    )

## Running a Chain on a Traced Dataset

Once you have a dataset, you can run a compatible chain or other object over it to see its results. The run traces will automatically be associated with the dataset for easy attribution and analysis.

**First, we'll define the chain we wish to run over the dataset.**

In this case, we're using an agent, but it can be any simple chain.

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

llm = ChatOpenAI(temperature=0)
tools = load_tools(['serpapi', 'llm-math'], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False)

**Now we're ready to run the chain!**

The docstring below hints ways you can configure the method to run.

In [11]:
?client.arun_on_dataset

Signature:
client.arun_on_dataset(
    dataset_name: 'str',
    llm_or_chain_factory: 'MODEL_OR_CHAIN_FACTORY',
    *,
    concurrency_level: 'int' = 5,
    num_repetitions: 'int' = 1,
    session_name: 'Optional[str]' = None,
    verbose: 'bool' = False,
) -> 'Dict[str, Any]'
Docstring:
Run the chain on a dataset and store traces to the specified session name.

Args:
    dataset_name: Name of the dataset to run the chain on.
    llm_or_chain_factory: Language model or Chain constructor to run
        over the dataset. The Chain constructor is used to permit
        independent calls on each example without carrying over state.
    concurrency_level: The number of async tasks to run concurrently.
    num_repetitions: Number of times to run the model on each example.
        This is useful when testing success rates or generating confidence
        intervals.
    session_name: Name of the session to store the traces in.
        Defaults to {dataset_name}-{chain class name}-{datetime}.
 

In [12]:
# Since chains can be stateful (e.g. they can have memory), we need provide
# a way to initialize a new chain for each row in the dataset. This is done
# by passing in a factory function that returns a new chain for each row.
chain_factory = lambda: initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    # We will use the intermediate steps later for evaluation
    return_intermediate_steps=True,
)

# If your chain is NOT stateful, your lambda can return the object directly
# to improve runtime performance. For example:
# chain_factory = lambda: agent

In [13]:
evaluation_session_name = "Search + Calculator Agent Evaluation"
chain_results = await client.arun_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=chain_factory,
    concurrency_level=5, # Optional, sets the number of examples to run at a time
    verbose=True,
    session_name=evaluation_session_name # Optional, a unique session name will be generated if not provided
)

# Sometimes, the agent will error due to parsing issues, incompatible tool inputs, etc.
# These are logged as warnings here and captured as errors in the tracing UI.

### Reviewing the Chain Results

You can review the results of the run in the tracing UI below and navigating to the session 
with the title **"Search + Calculator Agent Evaluation"**

In [14]:
# You can navigate to the UI by clicking on the link below
client

LangChainPlusClient (API URL: http://localhost:8000)

## Running an Evaluation Chain

Manually comparing the results of chains in the UI is effective, but it can be time consuming.
It's easier to leverage AI-assisted feedback to evaluate your agent's performance.

A few ways of doing this include:
- Adding ground-truth answers as outputs to the dataset and evaluating relative to those references.
- Evaluating the overall agent trajectory based on the tool usage and intermediate steps.
- Evaluating performance based on 'context' such as retrieved documents or tool results.
- Evaluating 'aspects' of the agent's response in a reference-free manner using targeted agent prompts.
    
We will demonstrate the first two here. First, we will supply labels to the dataset.

**Note: the feedback API is currently experimental and subject to change.**

In [15]:
import pandas as pd
from langchain.evaluation.loading import load_dataset

dataset = load_dataset("agent-search-calculator")
df = pd.DataFrame(dataset, columns=["question", "answer"])

Found cached dataset json (/Users/wfh/.cache/huggingface/datasets/LangChainDatasets___json/LangChainDatasets--agent-search-calculator-8a025c0ce5fb99d2/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
examples = client.list_examples(dataset_name=dataset_name)
examples_dict = {
    example.inputs['input']: example.id for example in examples
}
for tup in df.itertuples():
    if tup.question not in examples_dict:
        continue
    example_id = examples_dict[tup.question]
    client.update_example(outputs={"answer": tup.answer}, example_id=example_id)

In [17]:
from langchain.evaluation.qa import QAEvalChain

eval_llm = ChatOpenAI(model="gpt-4")
chain = QAEvalChain.from_llm(eval_llm)

In [18]:
# Fetch the session that was auto-generated
latest_session = client.read_session(session_name=evaluation_session_name)
runs = list(client.list_runs(session_id=latest_session.id, execution_order=1, error=False))

In [19]:
# Prepare inputs
answers = [client.read_example(example_id=run.reference_example_id).outputs["answer"] for run in runs]
inputs = [{**run.inputs, "answer": answer} for run, answer in zip(runs, answers)]
outputs = [{"output": run.outputs["output"]} for run in runs]

In [20]:
results = chain.evaluate(
    examples = inputs,
    predictions=outputs,
    question_key="input",
    answer_key="answer",
    prediction_key="output",
)

for run, result in zip(runs, results):
    client.create_feedback(run.id, "grade", result["text"], source_info={"evaluator": "QAEvalChain"})

## Evaluating Agent Trajectories

Often ground-truth reference labels aren't available to grade your
model's performance. In this case, there are a variety of "reference-free" evaluation
techniques that can be used.

One of these is the `TrajectoryEvalChain`, which scores the agent based on logical use
of its tools and how helpful the ultimate answer is.

This is just one example of how to you can evaluate your agent's behavior, and we encourage
you to develop your own chains to grade your agent along other dimensions.

In [21]:
from langchain.evaluation.agents import TrajectoryEvalChain

eval_llm = ChatOpenAI(temperature=0, model_name="gpt-4")

eval_chain = TrajectoryEvalChain.from_llm(
    llm=eval_llm,
    agent_tools=agent.tools,
    return_reasoning=True,
)

In [22]:
# Fetch the session used for the latest runs
latest_session = client.read_session(session_name=evaluation_session_name)
runs = client.list_runs(session_id=latest_session.id, execution_order=1, error=False)

In [24]:
from langchain.schema import AgentAction
from tqdm.notebook import tqdm

def get_intermediate_steps(run):
    """Convert the serialized steps into an object."""
    results = []
    for action, observation in run.outputs['intermediate_steps']:
        results.append([AgentAction(*action), observation])
    return results

for run in tqdm(runs):
    evaluation = eval_chain(
        inputs={
            "question": run.inputs['input'],
            "answer": run.outputs['output'],
            "agent_trajectory": eval_chain.get_agent_trajectory(get_intermediate_steps(run))
        },
    )
    client.create_feedback(
        run.id, 
        "trajectory_evaluation",
        evaluation["score"], 
       source_info={
           "evaluation_model": "gpt-4",
            "evaluation_chain": "TrajectoryEvalChain"
       }
    )

0it [00:00, ?it/s]

**Now the evaluation feedback is saved for future analysis.**

## Running a Chat Model over a Traced Dataset

We've shown how to run a _chain_ over a dataset, but you can also run an LLM or Chat model over a datasets formed from runs. 

First, we'll show an example using a ChatModel. This is useful for things like:
- Comparing results under different decoding parameters
- Comparing model providers
- Testing for regressions in model behavior
- Running multiple times with a temperature to gauge stability 

To speed things up, we'll upload a dataset we've previously captured directly to the tracing service.

In [25]:
import pandas as pd
from langchain.evaluation.loading import load_dataset

chat_dataset = load_dataset("two-player-dnd")
chat_df = pd.DataFrame(chat_dataset)
chat_df.head()

Found cached dataset parquet (/Users/wfh/.cache/huggingface/datasets/LangChainDatasets___parquet/LangChainDatasets--two-player-dnd-cc62c3037e2d9250/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

,generations,messages
0,"[[{'generation_info': None, 'message': {'conte...",[{'data': {'content': 'Here is the topic for a...
1,"[[{'generation_info': None, 'message': {'conte...",[{'data': {'content': 'Here is the topic for a...
2,"[[{'generation_info': None, 'message': {'conte...",[{'data': {'content': 'Here is the topic for a...
3,"[[{'generation_info': None, 'message': {'conte...",[{'data': {'content': 'Here is the topic for a...
4,"[[{'generation_info': None, 'message': {'conte...",[{'data': {'content': 'Here is the topic for a...


In [26]:
chat_dataset_name = "two-player-dnd"

if chat_dataset_name not in set([dataset.name for dataset in client.list_datasets()]):
    client.upload_dataframe(chat_df, 
                            name=chat_dataset_name,
                            description="An example dataset traced from chat models in a multiagent bidding dialogue",
                            input_keys=["messages"],
                            output_keys=["generations"],
                   )

### Reviewing behavior with temperature

Here, we will set `num_repetitions > 1` and set the temperature to 0.3 to see the variety of response types for a each example.


In [29]:
# !pip install anthropic > /dev/null

In [30]:
from langchain.chat_models import ChatAnthropic

chat_model = ChatAnthropic(temperature=.3)

In [31]:
chat_model_results = await client.arun_on_dataset(
    dataset_name=chat_dataset_name,
    llm_or_chain_factory=chat_model,
    concurrency_level=5, # Optional, sets the number of examples to run at a time
    num_repetitions=3,
    verbose=True
)

# The 'experimental tracing v2' warning is expected, as we are still actively developing the v2 tracing API 
# Since we are running examples concurrently,  you may run into some RateLimit warnings from your model
# provider. In most cases, the tests will still run to completion (the wrappers have backoff).

### Reviewing the Chat Model Results

You can review the latest runs by clicking on the link below and navigating to the "two-player-dnd" session.

In [32]:
client

LangChainPlusClient (API URL: http://localhost:8000)

## Running an LLM over a Traced Dataset

You can run an LLM over a dataset in much the same way as the chain and chat models, provided the dataset you've captured is in the appropriate format. We've cached one for you here, but using application-specific traces will be much more useful for your use cases.

In [33]:
from langchain.llms import OpenAI

llm = OpenAI(model_name='text-curie-001', temperature=0)

In [34]:
completions_dataset = load_dataset("state-of-the-union-completions")
completions_df = pd.DataFrame(completions_dataset)
completions_df.head()

Found cached dataset parquet (/Users/wfh/.cache/huggingface/datasets/LangChainDatasets___parquet/LangChainDatasets--state-of-the-union-completions-5347290a406c64c8/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

,generations,ground_truth,prompt
0,[[{'generation_info': {'finish_reason': 'stop'...,The pandemic has been punishing. \n\nAnd so ma...,"Putin may circle Kyiv with tanks, but he will ..."
1,[[]],With a duty to one another to the American peo...,"Madam Speaker, Madam Vice President, our First..."
2,[[{'generation_info': {'finish_reason': 'stop'...,He thought he could roll into Ukraine and the ...,With a duty to one another to the American peo...
3,[[]],And the costs and the threats to America and t...,"Please rise if you are able and show that, Yes..."
4,[[{'generation_info': {'finish_reason': 'stop'...,"Please rise if you are able and show that, Yes...",Groups of citizens blocking tanks with their b...


In [35]:
completions_dataset_name = "state-of-the-union-completions"

if completions_dataset_name not in set([dataset.name for dataset in client.list_datasets()]):
    client.upload_dataframe(completions_df, 
                            name=completions_dataset_name,
                            description="An example dataset traced from completion endpoints over the state of the union address",
                            input_keys=["prompt"],
                            output_keys=["generations"],
                   )

In [36]:
# We also offer a synchronous method for running examples if a chain or llm's async methods aren't yet implemented
completions_model_results = client.run_on_dataset(
    dataset_name=completions_dataset_name,
    llm_or_chain_factory=llm,
    num_repetitions=1,
    verbose=True
)

## Reviewing the LLM Results

You can once again inspect the latest runs by clicking on the link below and navigating to the "two-player-dnd" session.

In [37]:
client

LangChainPlusClient (API URL: http://localhost:8000)